In [1]:
import os

import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this

C:\Users\Chankoo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-NLP'

In [3]:
os.chdir('./../airbnb-data')

In [5]:
review ={}
import json
with open('pos_review.json','r',encoding = 'utf-8') as fp:
    review.update(json.load(fp))

In [6]:
texts = []
for home_id,rev_lst in review.items():
    for rev in rev_lst:
        for sentence in rev[1] :
            texts.append(sentence)

In [8]:
dic = corpora.Dictionary(texts)

In [10]:
dic.save('dic_review.dict')

In [11]:
test_dic = open('dic_review.dict','r')

In [28]:
corpus_vector = [dic.doc2bow(text) for text in texts]

In [15]:
corpora.MmCorpus.serialize('corpus_review.mm', corpus_vector)  # store to disk, for later use

### gensim.models.ldamodel.LdaModel parameter

- corpus: the corpus used to train the topic model.

- id2word: a dictionary that maps word numerical identifiers to word strings (see e.g. this tutorial for an explanation). In our case, this is handled automatically by the corpus class.

- alpha: the topic distribution. If we set it to 'auto', then we tell the algorithm to determine the topic proportions automatically.

- num_topics: how many topics the algorithm should find.

- passes: how many times the algorithm should go through the corpus. If you are using a large corpus (e.g. Wikipedia), then this can be set to 1, but for a small corpus (e.g. the reviews) then it should be a larger value.

In [10]:
K=10

In [35]:
%%time
model = gensim.models.ldamodel.LdaModel(
                        corpus_vector, 
                        num_topics=K, 
                        id2word = dic, 
                        passes=10,
                        eval_every=5
                        ) #LDA model 생성

Wall time: 2h 13min 9s


In [18]:
# dic_review = {}
# dic_review.update(dic)

In [36]:
# list(dic_review.items())[0]

In [18]:
# with open('dic_review.json','w',encoding='utf-8') as fp:
#     json.dump(dic_review,fp,ensure_ascii=False)

In [37]:
model.show_topics(num_words=10) #토픽을 구성하는 주요단어들

[(0,
  '0.075*"station_N" + 0.036*"convenient_N" + 0.033*"location_N" + 0.031*"walk_V" + 0.026*"subway_N" + 0.024*"place_N" + 0.024*"minute_N" + 0.023*"easy_J" + 0.020*"locate_V" + 0.020*"store_N"'),
 (1,
  '0.100*"good_J" + 0.070*"nice_J" + 0.065*"location_N" + 0.065*"really_R" + 0.054*"place_N" + 0.034*"comfortable_J" + 0.034*"clean_J" + 0.026*"stay_N" + 0.025*"house_N" + 0.016*"accommodation_N"'),
 (2,
  '0.067*"area_N" + 0.036*"night_N" + 0.030*"quiet_J" + 0.024*"quite_R" + 0.023*"view_N" + 0.023*"price_N" + 0.022*"bit_N" + 0.020*"see_V" + 0.020*"neighborhood_N" + 0.018*"right_J"'),
 (3,
  '0.151*"great_J" + 0.038*"home_N" + 0.035*"experience_N" + 0.033*"thank_N" + 0.023*"stay_N" + 0.018*"make_V" + 0.017*"thing_N" + 0.015*"airbnb_N" + 0.015*"place_V" + 0.015*"feel_N"'),
 (4,
  '0.112*"room_N" + 0.052*"well_R" + 0.051*"everything_N" + 0.051*"clean_J" + 0.025*"house_N" + 0.024*"apartment_N" + 0.022*"need_V" + 0.022*"clean_N" + 0.021*"small_J" + 0.019*"comfortable_J"'),
 (5,
  '0.024*

In [14]:
topic_keywords = {}
for i in range(K):
    topic_keywords[i] = model.show_topic(i,topn=30)

In [15]:
topic_keywords

{0: [('station_N', 0.075440474),
  ('convenient_N', 0.035710342),
  ('location_N', 0.033328794),
  ('walk_V', 0.03065599),
  ('subway_N', 0.026240626),
  ('place_N', 0.023897689),
  ('minute_N', 0.023682337),
  ('easy_J', 0.022524938),
  ('locate_V', 0.019995727),
  ('store_N', 0.019815892),
  ('close_J', 0.019692997),
  ('close_R', 0.019388903),
  ('away_R', 0.01791159),
  ('bus_N', 0.017175121),
  ('airport_N', 0.0151694175),
  ('convenience_N', 0.014715803),
  ('also_R', 0.014201972),
  ('walk_N', 0.013559201),
  ('subway_J', 0.01225483),
  ('apartment_N', 0.01212426),
  ('convenient_J', 0.01193456),
  ('go_V', 0.010800422),
  ('distance_N', 0.010039834),
  ('find_V', 0.009765462),
  ('subway_R', 0.009758889),
  ('take_V', 0.009583443),
  ('get_V', 0.009291667),
  ('line_N', 0.009158479),
  ('restaurant_N', 0.0081279725),
  ('house_N', 0.007849644)],
 1: [('good_J', 0.09984607),
  ('nice_J', 0.0702112),
  ('location_N', 0.065345466),
  ('really_R', 0.06501797),
  ('place_N', 0.05364

In [41]:
# model.save('ldamodel_t10_p10.model')

In [7]:
model = gensim.models.ldamodel.LdaModel.load('ldamodel_t10_p10.model')

In [ ]:
# Compute Perplexity 
print('\nPerplexity: ', ldamodel.log_perplexity(corpus_vector))  # a measure of how good the model is. lower the better.

In [43]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus_vector, dic)
vis

C:\Users\Chankoo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.081128 -0.100378       1        1  12.661897
1      0.171820 -0.161947       2        1  11.578853
4     -0.032812 -0.111745       3        1  11.465013
8      0.254983  0.175856       4        1  10.728833
9     -0.197459 -0.070546       5        1  10.380273
6      0.011459  0.298220       6        1  10.177848
5     -0.282095  0.136767       7        1   9.406765
7     -0.010544 -0.236194       8        1   8.278798
2     -0.127053  0.011373       9        1   7.811550
3      0.130571  0.058595      10        1   7.510164, topic_info=      Category           Freq           Term          Total  loglift  logprob
term                                                                         
14     Default  137654.000000        great_J  137654.000000  30.0000  30.0000
9      Default  156123.000000         room_N  156123.000000  29.0000  29.0000
25     Default  166305.000000         good_J  166305.000000  28.0000  28.0000
106    Default  234423.000000        place_N  234423.000000  27.0000  27.0000
60     Default  123868.000000         host_N  123868.000000  26.0000  26.0000
230    Default  115914.000000      station_N  115914.000000  25.0000  25.0000
70     Default  143025.000000     location_N  143025.000000  24.0000  24.0000
105    Default  124074.000000         nice_J  124074.000000  23.0000  23.0000
75     Default  115082.000000       really_R  115082.000000  22.0000  22.0000
37     Default  118413.000000        clean_J  118413.000000  21.0000  21.0000
92     Default  104745.000000         stay_V  104745.000000  20.0000  20.0000
76     Default   63351.000000         area_N   63351.000000  19.0000  19.0000
283    Default  123966.000000         stay_N  123966.000000  18.0000  18.0000
295    Default   72051.000000         well_R   72051.000000  17.0000  17.0000
10     Default   97863.000000         time_N   97863.000000  16.0000  16.0000
314    Default   71433.000000   everything_N   71433.000000  15.0000  15.0000
146    Default   54957.000000          day_N   54957.000000  14.0000  14.0000
151    Default   54573.000000    recommend_V   54573.000000  13.0000  13.0000
216    Default   54869.000000   convenient_N   54869.000000  12.0000  12.0000
56     Default   74406.000000  comfortable_J   74406.000000  11.0000  11.0000
141    Default   43995.000000          lot_N   43995.000000  10.0000  10.0000
111    Default   47543.000000         come_V   47543.000000   9.0000   9.0000
383    Default   47490.000000   definitely_R   47490.000000   8.0000   8.0000
341    Default   53491.000000   restaurant_N   53491.000000   7.0000   7.0000
288    Default   59309.000000         make_V   59309.000000   6.0000   6.0000
128    Default   43177.000000         kind_N   43177.000000   5.0000   5.0000
20     Default   47103.000000         walk_V   47103.000000   4.0000   4.0000
282    Default   58085.000000        night_N   58085.000000   3.0000   3.0000
17     Default   40571.000000         next_J   40571.000000   2.0000   2.0000
165    Default   34979.000000         home_N   34979.000000   1.0000   1.0000
...        ...            ...            ...            ...      ...      ...
404    Topic10   10854.346680  instruction_N   10855.244141   2.5888  -4.4304
625    Topic10    8298.721680         know_V    8299.619141   2.5888  -4.6988
1106   Topic10    8233.368164         flat_J    8234.265625   2.5888  -4.7067
877    Topic10    7350.484375          ive_J    7351.382324   2.5888  -4.8202
1691   Topic10    6941.799316     anything_N    6942.697266   2.5888  -4.8774
211    Topic10    6855.778809         meet_N    6856.676758   2.5888  -4.8898
954    Topic10    6514.379395        value_N    6515.277344   2.5888  -4.9409
1451   Topic10    6348.928223         book_V    6349.826172   2.5888  -4.9666
1816   Topic10    6177.219238       choice_N    6178.117188   2.5888  -4.9941
277    Topic10    6162.947266    